In [1]:
import os
os.chdir('../')

In [2]:
!set MLFLOW_TRACKING_URI=https://dagshub.com/provezano/end-to-end-ml-project.mlflow
!set MLFLOW_TRACKING_USERNAME=provezano
!set MLFLOW_TRACKING_PASSWORD=c54ccb19ca6b05fb3141d21e684fc1c894086d7a
#python script.py

In [3]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/provezano/end-to-end-ml-project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='provezano'
os.environ['MLFLOW_TRACKING_PASSWORD']='c54ccb19ca6b05fb3141d21e684fc1c894086d7a'
#python script.py

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    #train_data_path: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
            self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = 'https://dagshub.com/provezano/end-to-end-ml-project.mlflow'
        )
        return model_evaluation_config

In [6]:
import os
import mlflow
import joblib
import numpy as np
import pandas as pd
import mlflow.sklearn
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_squared_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)

        

        with mlflow.start_run():
            print('artifact uri:', mlflow.get_artifact_uri())
            tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme
            
            predicted_qualities = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {'rmse': rmse, 'mae': mae, 'r2': r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric('rmse', rmse)
            mlflow.log_metric('r2', r2)
            mlflow.log_metric('mae', mae)

            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model, 'model', registered_model_name='ElasticnetModel')
            else:
                mlflow.sklearn.log_model(model, 'model')


[2024-01-20 12:01:12,074: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-01-20 12:01:12,075: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-20 12:01:13,916: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-20 12:01:13,919: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-20 12:01:13,922: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-20 12:01:13,923: INFO: common: created directory at: artifacts]
[2024-01-20 12:01:13,924: INFO: common: created directory at: artifacts/model_evaluation]
artifact uri: mlflow-artifacts:/00b17062ac984815bce39f42d9223f82/b4349cb1313b43d0a92be3a0b0bfad74/artifacts
[2024-01-20 12:01:15,899: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/01/20 12:01:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.
